In [36]:
import requests  # Para peticiones HTTP
import json      # Para trabajar con JSON
import pandas as pd  # Para DataFrames
from numpy import nan
from IPython.display import display  # Para mostrar resultados en Jupyter

# Tu API key de RapidAPI para AeroDataBox
flights_key = "6398d16efdmshfb022c9141871dcp15f1b7jsn115de15bb173"


In [37]:
# URL de la API para vuelos de llegada a Casablanca (ICAO: GMMN)
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/GMMN/2025-09-01T08:00/2025-09-01T20:00"

# Parámetros de la API
querystring = {
    "withLeg": "true",
    "direction": "Arrival",
    "withCancelled": "true",
    "withCodeshared": "true",
    "withCargo": "true",
    "withPrivate": "true",
    "withLocation": "false"
}

# Headers con tu API key
headers = {
    "x-rapidapi-host": "aerodatabox.p.rapidapi.com",
    "x-rapidapi-key": flights_key
}


In [38]:
response = requests.get(url, headers=headers, params=querystring)

if response.status_code == 200:
    flights_data = response.json()
    
    # Guardar en data.json
    try:
        with open("data.json", "r") as f:
            existing_data = json.load(f)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(flights_data)

    with open("data.json", "w") as f:
        json.dump(existing_data, f, indent=4)
        print("✅ Data appended to data.json file.")
else:
    print("❌ Failed to retrieve data from the API. Status code:", response.status_code)


✅ Data appended to data.json file.


In [39]:
# Guardar los vuelos de llegada
flight_arrivals = flights_data.get("arrivals", [])

# Crear lista con los campos importantes
flights_list = []
for flight in flight_arrivals:
    departure_airport = flight.get("departure", {}).get("airport", {})
    departure_country = departure_airport.get("country", {}).get("iataCode", nan)
    
    arrival_info = flight.get("arrival", {}).get("scheduledTime", {})
    
    flight_info = {
        "flight_number": flight.get("number", nan),
        "from": departure_airport.get("name", nan),
        "from_country": departure_country,
        "to": "Casablanca",  # Destino fijo
        "to_country": "MA",  # País fijo
        "airline": flight.get("airline", {}).get("name", nan),
        "aircraft": flight.get("aircraft", {}).get("model", nan),
        "Scheduled_arrival_time.utc": arrival_info.get("utc", nan),
        "Scheduled_arrival_time.local": arrival_info.get("local", nan)
    }
    flights_list.append(flight_info)



In [40]:
# Crear DataFrame
df_flights = pd.DataFrame(flights_list)

# Mostrar las primeras filas
display(df_flights.head())



,flight_number,from,from_country,to,to_country,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,AT 512,Lomé,NaN,Casablanca,MA,Royal Air Maroc,Boeing 737-800,2025-09-01 07:20Z,2025-09-01 08:20+01:00
1,AT 1401,Oujda,NaN,Casablanca,MA,Royal Air Maroc,Boeing 737-800 (winglets),2025-09-01 07:25Z,2025-09-01 08:25+01:00
2,AT 1447,Errachidia,NaN,Casablanca,MA,Royal Air Maroc,Embraer 190,2025-09-01 07:30Z,2025-09-01 08:30+01:00
3,AT 558,Cotonou,NaN,Casablanca,MA,Royal Air Maroc,Boeing 737-800,2025-09-01 07:40Z,2025-09-01 08:40+01:00
4,AT 219,Washington,NaN,Casablanca,MA,Royal Air Maroc,Boeing 787-8,2025-09-01 08:20Z,2025-09-01 09:20+01:00
